# Introduction
The objective of this report is to evaluate the assumption that cows face straight ahead while walking, using data from GPS and IMU sensors on cow collars. The data consisted of timestamped records of latitude, longitude, GPS heading, IMU heading, and GPS speed for 14 cows walking down a raceway.

# Part 1: Solve two main task

## Data Preprocessing

### Merge two data set together and Adjusted IMU headings to a 0-360 degree range.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings

warnings.filterwarnings("ignore")
gps_data = pd.read_csv('/content/gps_data.csv')
imu_data = pd.read_csv('/content/imu_data.csv')
missing_GPSvalues = gps_data.isnull().sum()
print(missing_GPSvalues)
missing_imuvalues = imu_data.isnull().sum()
print(missing_imuvalues)
# Merge based on both Serial Number and NZDT (Mutiple selection by '[]')
merged_data = pd.merge(gps_data, imu_data, on=['Serial Number','NZDT'])
merged_data['NZDT'] = pd.to_datetime(merged_data['NZDT'], format='%d/%m/%y %H:%M:%S')
def angle_standard(angle1,angle2):
    res = np.abs(angle1-angle2)%360
    return np.where(res > 180, 360 - res, res)
merged_data["difference"] = angle_standard(merged_data['Heading'], merged_data['GPS Heading'])

Serial Number    0
NZDT             0
Latitude         0
Longitude        0
GPS Heading      0
GPS Speed        0
dtype: int64
Serial Number    0
NZDT             0
Heading          0
dtype: int64


There have no missing value with in both imu data and GPS data.

## Task 1: Investage if cows are truly face straight while walking.
I'll solve this task through statistical analysis:

### Setting Hypothesis

H0: There have no difference between the IMU and GPS headings.

Ha: There have difference between the IMU and GPS heading.

### Statistics Summary

In [3]:
merged_data["difference"].describe()

count    3309.000000
mean       16.991542
std        12.030533
min         0.006267
25%         7.721167
50%        16.486700
75%        24.024919
max       138.019127
Name: difference, dtype: float64

*   The mean heading difference is approximately 17 degrees, suggesting that while cows generally face forward, there is variability.
*   The maximum heading difference observed is around 138 degrees, indicating significant deviations exist.

### T-test

Now with the shown summary of their difference, mean difference is 16.99 and standard deviation of difference is 12.03, preset significance level α = 0.01

In [4]:
from scipy import stats
sd = 12.03
mean = 16.99
standard_error = sd / np.sqrt(3309)
print(f"Standard Error is:{standard_error}")
#μ0 (difference between two group) = 0
t_stastistic = (mean - 0) / standard_error
print(f"T_stastistics is :{t_stastistic}" )
df = 3308
print(f"Degree of freedom is :{df}" )
p_value = 2 * (1 - stats.t.cdf(abs(81.246), df=3308))
print(f"P-value is :{p_value}" )

Standard Error is:0.20913043642161855
T_stastistics is :81.241163604456
Degree of freedom is :3308
P-value is :0.0


Now for a two-tailed test with α = 0.01 and df = 3308, the critical t-value is approximately ±2.576, and the T-statistic value is much larger than it. And the p-value is very close to 0 and proves it smaller than 0.01. So I can reject H0 hypothesis, this strong statistical evidence shows there are significant difference between the IMU and GPS heading. It means the assumption that cows face straight ahead while walking is not a reasonable assumption.

# Task2： Show how valid this claim is
I'll make a dashboard to tell the difference, and check the anomalies.

In [5]:
%%capture
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.5 MB/s eta 0:00:00


In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go

# Now do something fancy
merged_data['Heading'] = merged_data['Heading'] % 360
merged_data['GPS Heading'] = merged_data['GPS Heading'] % 360
map_center = {"lat": merged_data['Latitude'].mean(), "lon": merged_data['Longitude'].mean()}
zoom_level = 20

merged_data['Adjusted GPS Heading'] = merged_data['Heading'] + merged_data['difference']
merged_data['Adjusted GPS Heading'] = merged_data['Adjusted GPS Heading'] % 360

def smooth_heading_transition(heading_series):
    adjusted_heading = heading_series.copy()
    for i in range(1, len(heading_series)):
        if abs(adjusted_heading[i] - adjusted_heading[i - 1]) > 180:
            if adjusted_heading[i] > adjusted_heading[i - 1]:
                adjusted_heading[i] -= 360
            else:
                adjusted_heading[i] += 360
    return adjusted_heading

merged_data['Adjusted Heading'] = smooth_heading_transition(merged_data['Heading'])
merged_data['Adjusted GPS Heading'] = smooth_heading_transition(merged_data['Adjusted GPS Heading'])

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Cow Movement Analysis Dashboard"),
        dcc.Graph(
        id='map-graph',
        figure=px.scatter_mapbox(
            merged_data,
            lat='Latitude',
            lon='Longitude',
            color='Serial Number',
            mapbox_style="carto-positron",
            title='Cow Movements'
        ).update_layout(
            mapbox=dict(
                center=map_center,
                zoom=zoom_level
            )
        )
    ),

    dcc.Graph(
        id='difference-histogram',
        figure=px.histogram(
            merged_data,
            x='difference',
            nbins=30,
            title='Histogram of Heading Differences'
        )
    ),

    dcc.Graph(
        id='anomaly-graph',
        figure=go.Figure(
            data=[
                go.Scatter(
                    x=merged_data['NZDT'],
                    y=merged_data['difference'],
                    mode='markers',
                    marker=dict(
                        color=['red' if diff > 60 else 'blue' for diff in merged_data['difference']]
                    ),
                    name='Heading Difference'
                )
            ],
            layout=go.Layout(
                title='Heading Differences with Anomalies Highlighted'
            )
        )
    ),
    dcc.Dropdown(
        id='cow-selector',
        options=[{'label': serial, 'value': serial} for serial in merged_data['Serial Number'].unique()],
        value=merged_data['Serial Number'].unique()[0]
    ),
    dcc.Graph(id='heading-plot'),
    dcc.Graph(id='histogram-plot'),
    html.Div(id='anomaly-frequency')
])

@app.callback(
    Output('heading-plot', 'figure'),
    Input('cow-selector', 'value')
)
def update_heading_plot(selected_cow):
    cow_data = merged_data[merged_data['Serial Number'] == selected_cow]
    figure = {
        'data': [
            go.Scatter(
                x=cow_data['NZDT'], y=cow_data['Adjusted Heading'], mode='lines', name='Heading (IMU)'
            ),
            go.Scatter(
                x=cow_data['NZDT'], y=cow_data['Adjusted GPS Heading'], mode='lines', name='GPS Heading'
            )
        ],
        'layout': go.Layout(
            title=f'Time Series of Adjusted Headings for Cow {selected_cow}',
            xaxis={'title': 'Time'},
            yaxis={'title': 'Adjusted Heading'},
            hovermode='closest'
        )
    }
    return figure

@app.callback(
    Output('histogram-plot', 'figure'),
    Input('cow-selector', 'value')
)
def update_histogram(selected_cow):
    cow_data = merged_data[merged_data['Serial Number'] == selected_cow]
    if cow_data.empty:
        return {
            'data': [],
            'layout': go.Layout(
                title='No data available',
                xaxis={'title': 'Heading Difference'},
                yaxis={'title': 'Frequency'},
                hovermode='closest'
            )
        }
    figure = {
        'data': [
            go.Histogram(
                x=cow_data['difference'], nbinsx=30, name='Heading Difference'
            )
        ],
        'layout': go.Layout(
            title=f'Distribution of Heading Differences for Cow {selected_cow}',
            xaxis={'title': 'Heading Difference'},
            yaxis={'title': 'Frequency'},
            hovermode='closest'
        )
    }
    return figure
@app.callback(
    Output('anomaly-frequency', 'children'),
    Input('cow-selector', 'value')
)
def update_anomaly_frequency(selected_cow):
    cow_data = merged_data[merged_data['Serial Number'] == selected_cow]
    if cow_data.empty:
        return f'Number of Anomalies for Cow {selected_cow}: 0'
    mean_diff = cow_data['difference'].mean()
    std_diff = cow_data['difference'].std()
    threshold_upper = mean_diff + 3 * std_diff
    threshold_lower = mean_diff - 3 * std_diff
    anomalies = cow_data[(cow_data['difference'] > threshold_upper) | (cow_data['difference'] < threshold_lower)]
    anomaly_count = anomalies.shape[0]
    return f'Number of Anomalies for Cow {selected_cow}: {anomaly_count}'

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

From the histogram we can know that most heading differences fall within a range of 0 to 25 degrees, with a few outliers having larger differences.

## Anomalies and potential sources of error

Setting threshold that if difference greater than 30 degrees then it is a anomaly movement

In [8]:
anomalies = merged_data[merged_data['difference'] > 60]
anomalies.head()

,Serial Number,NZDT,Latitude,Longitude,GPS Heading,GPS Speed,Heading,difference,Adjusted GPS Heading,Adjusted Heading
95,004-0009-00481,2020-10-16 17:25:16,-37.554699,175.459136,345,0.036925,278.758151,66.241849,-15.0,-81.241849
96,004-0009-00481,2020-10-16 17:25:14,-37.554699,175.459136,345,0.062614,269.011829,75.988171,-15.0,-90.988171
97,004-0009-00481,2020-10-16 17:25:13,-37.554699,175.459135,345,0.041649,275.817974,69.182026,-15.0,-84.182026
98,004-0009-00481,2020-10-16 17:25:12,-37.554699,175.459134,345,0.259587,279.595621,65.404379,-15.0,-80.404379
241,004-0009-00431,2020-10-16 16:47:57,-37.553478,175.459150,17,0.565000,301.988220,75.011780,17.0,-58.011780


Potential Sources of Error:
*   Sensor Broken:
The IMU or GPS sensors might occasionally broken, leading to inaccurate readings.
*   Environmental Interference:
External factors like magnetic interference or poor GPS signal reception can affect sensor accuracy.
*   Movement Dynamics:
Sudden movements or changes in the cow's direction might cause discrepancies between the IMU and GPS readings.

Recommendations for Improvement:


*   Sensor Calibration:
Try to do the calibration of IMU and GPS sensors to keep accuracy
*   Introduce new variables:
Introduce new variables like weather or cow's health status. to help on cross-verify result.
*   Try to refine the algorithms used for calculating headings and handling edge cases where sudden movements occur like your Halter description required.





## Get closer to anomalies and try to find if we can build a model to make more accurate prediction

Ok, what if we can analysis anomalies and and try to find hidden pattern inside of it? That must be a strong support for me to build trustable prediction model that can calculate truly heading based on the GPS data. First,get all anomaly for all cows.

In [9]:
def analyze_anomalies_by_serial(anomalies, data, time_window='10T'):
    results = {}
    for serial in anomalies['Serial Number'].unique():
        serial_anomalies = anomalies[anomalies['Serial Number'] == serial]
        serial_results = []
        for idx, anomaly in serial_anomalies.iterrows():
            start_time = anomaly['NZDT'] - pd.Timedelta(time_window)
            end_time = anomaly['NZDT'] + pd.Timedelta(time_window)
            context_data = data[(data['NZDT'] >= start_time) & (data['NZDT'] <= end_time) & (data['Serial Number'] == serial)]
            serial_results.append((anomaly['NZDT'], context_data))
        results[serial] = serial_results
    return results

anomaly_contexts_by_serial = analyze_anomalies_by_serial(anomalies, merged_data)


Now I got every anomalies, and I want to find the hidden pattern behind the anomaly data, and the data is based on time-series. So I'll go introduce ARIMA models.

ARIMA, which stands for AutoRegressive Integrated Moving Average, is a method used to analyze and forecast time series data. It looks at patterns in past data to predict future values. ARIMA combines three parts: AutoRegressive (AR) terms that use past values, Integrated (I) terms that involve differencing the data to make it stable, and Moving Average (MA) terms that use past errors in predictions. This method helps in identifying trends and making accurate predictions over time.

The complexity of the ARIMA model needed to make predictions can reveal if the data quality is good or lacking. When consistent ARIMA models work well on all anomaly situation, it means the data is good and captures the main factors influencing cow movements. However, if different complex models are required to make prediction for different cows, it indicates that important information might be missing from the data. It will be a sign that current data might not be enough to make accurate predictions.

In [10]:
%%capture
!pip install pmdarima

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00


In [11]:
import pmdarima as pm
from pmdarima import auto_arima
# Function to check stationarity and apply differencing if necessary
def make_stationary(context_data):
    from statsmodels.tsa.stattools import adfuller
    p_value = adfuller(context_data['Heading'])[1]
    if p_value > 0.05:
        context_data['Heading'] = context_data['Heading'].diff().dropna()
    return context_data

def find_best_arima_model(context_data):
    context_data = context_data.copy()
    if 'NZDT' not in context_data.columns:
        print("Error: 'NZDT' column is missing from the context data")
        return None, None
    context_data.set_index('NZDT', inplace=True)
    context_data = make_stationary(context_data)
    try:
        model = pm.auto_arima(context_data['Heading'], seasonal=False, trace=False,
                              error_action='ignore', suppress_warnings=True, stepwise=True)
        return model.order, model
    except Exception as e:
        print(f"Error fitting ARIMA model: {e}")
        return None, None

# Function to apply ARIMA model to the time series data around each anomaly
def apply_arima_model(anomaly_contexts_by_serial):
    arima_results = {}
    for serial, contexts in anomaly_contexts_by_serial.items():
        serial_arima_results = []
        for anomaly_time, context_data in contexts:
            print(f"Processing serial {serial} at anomaly time {anomaly_time}")
            best_order, best_model = find_best_arima_model(context_data)
            if best_model is not None:
                forecast = best_model.predict(n_periods=10)
                forecast_index = pd.date_range(start=context_data.index[-1], periods=10, freq='T')
                forecast_series = pd.Series(forecast, index=forecast_index)
                serial_arima_results.append((anomaly_time, forecast_series, best_order))
            else:
                print(f"No suitable ARIMA model found for serial {serial} at {anomaly_time}")
        arima_results[serial] = serial_arima_results
    return arima_results

arima_results = apply_arima_model(anomaly_contexts_by_serial)

Processing serial 004-0009-00481 at anomaly time 2020-10-16 17:25:16
Processing serial 004-0009-00481 at anomaly time 2020-10-16 17:25:14
Processing serial 004-0009-00481 at anomaly time 2020-10-16 17:25:13
Processing serial 004-0009-00481 at anomaly time 2020-10-16 17:25:12
Processing serial 004-0009-00431 at anomaly time 2020-10-16 16:47:57
Processing serial 004-0009-00431 at anomaly time 2020-10-16 16:47:54
Processing serial 004-0008-00009 at anomaly time 2020-10-16 16:03:47
Processing serial 004-0008-00009 at anomaly time 2020-10-16 16:03:46
Processing serial 004-0008-00009 at anomaly time 2020-10-16 16:00:46
Processing serial 004-0008-00009 at anomaly time 2020-10-16 16:00:45
Processing serial 004-0008-00009 at anomaly time 2020-10-16 16:00:44
Processing serial 004-0008-00009 at anomaly time 2020-10-16 16:00:43
Processing serial 004-0007-00034 at anomaly time 2020-10-16 16:11:04
Processing serial 004-0007-00034 at anomaly time 2020-10-16 16:09:29
Processing serial 004-0007-00041 a

In [12]:
# Summarize ARIMA parameters
def summarize_arima_orders(arima_results):
    orders = []
    for serial, results in arima_results.items():
        for anomaly_time, forecast, best_order in results:
            orders.append(best_order)
    orders_df = pd.DataFrame(orders, columns=['p', 'd', 'q'])
    return orders_df

orders_df = summarize_arima_orders(arima_results)
print("ARIMA Orders Summary:")
print(orders_df)


ARIMA Orders Summary:
    p  d  q
0   0  0  1
1   0  0  1
2   0  0  1
3   0  0  1
4   0  0  1
5   0  0  1
6   1  0  2
7   1  0  2
8   1  0  2
9   1  0  2
10  1  0  2
11  1  0  2
12  0  0  3
13  0  0  3
14  0  0  1
15  0  0  1
16  0  0  1
17  0  0  1
18  0  0  1
19  0  0  1
20  0  1  1
21  0  0  1


The analysis of ARIMA model orders applied to cow movement data reveals significant insights into the patterns and complexities associated with anomalies. This shows the currently dataset can not support strong and trustable prediction model building. Predominantly, the simple ARIMA(1,0,0) model is sufficient for predicting cow movements in most cases, indicating that a straightforward approach of considering only the last movement often works well. However, there are instances where more complex models, such as ARIMA(2,1,3) and ARIMA(1,0,2), are necessary, suggesting that these anomalies are influenced by additional factors not captured by simpler models. This variability in model complexity underscores the presence of different underlying processes driving cow movements during anomalies.

This inconsistent of ARIMA paramaters can prove that the existing data can not fully capture all variables causing anomalies. Potential factor that will be needed could be environmental conditions, behavioral dynamics, or operational activities on the farm. To enhance the robustness of predictions, it is recommended to incorporate additional input factors. This could involve collecting data on weather conditions, terrain types, herd dynamics, feeding schedules, and other relevant factors. By integrating these additional data points, the predictive models can become more comprehensive, leading to more accurate and reliable predictions of cow movements, ultimately improving farm management and operational efficiency.